#### Saturday, January 20, 2024

https://github.com/PacktPublishing/Vector-Search-for-Practitioners-with-Elastic/blob/main/chapter2/mapping-and-indexing-vectors.ipynb

This all runs. 

In [ ]:
# !pip install transformers elasticsearch

In [1]:
import numpy as np
from transformers import AutoTokenizer, AutoModel
from elasticsearch import Elasticsearch
import torch

In [2]:
# # Define Elasticsearch connection with credentials
# es = Elasticsearch(
#     ['https://hostname:port'],
#     http_auth=('username', 'password'),
#     verify_certs=False
# )

from elasticsearch import Elasticsearch 

# Password for the 'elastic' user generated by Elasticsearch
# esHost = "https://172.19.0.3:9200"
esHost = "https://172.19.0.2:9200"

ELASTIC_PASSWORD = "*l-ncw4tr*UF-qzEybkq"

path2cert = "/elasticsearch/http_ca.crt"

esClient = Elasticsearch(esHost, ca_certs=path2cert, basic_auth=("elastic", ELASTIC_PASSWORD))

esClient.info()

ObjectApiResponse({'name': 'f8bb06ea76ae', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'JtIKr7BlTaWWORxN6-qEEw', 'version': {'number': '8.11.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '76013fa76dcbf144c886990c6290715f5dc2ae20', 'build_date': '2023-12-05T10:03:47.729926671Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [3]:
# Define the mapping for the dense vector field
mapping = {
    'properties': {
        'embedding': {
            'type': 'dense_vector',
            'dims': 768   # the number of dimensions of the dense vector
        }
    }
}


In [4]:

# Create an index with the defined mapping
esClient.indices.create(index='chapter-2', body={'mappings': mapping})

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'chapter-2'})

In [5]:
# Define a set of documents
docs = [
    {
        'title': 'Document 1',
        'text': 'This is the first document.'
    },
    {
        'title': 'Document 2',
        'text': 'This is the second document.'
    },
    {
        'title': 'Document 3',
        'text': 'This is the third document.'
    }
]

In [6]:
# Load the BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

In [7]:
# Generate embeddings for the documents using BERT
for doc in docs:
    text = doc['text']
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        output = model(**inputs).last_hidden_state.mean(dim=1).squeeze(0).numpy()
    doc['embedding'] = output.tolist()

In [8]:
# Index the documents in Elasticsearch
for doc in docs:
    esClient.index(index='chapter-2', body=doc)